# pydca demo

## MSA Trimming
Here we show how to use pydca as a library. We start with by importing pydca or selected modules within pydca.

In [2]:

"""pydca demo

Author: Mehari B. Zerihun
"""

# import pydca and/or selected modules
import pydca
from pydca.plmdca import plmdca
from pydca.meanfield_dca import meanfield_dca
from pydca.sequence_backmapper import sequence_backmapper
from pydca.msa_trimmer import msa_trimmer
from pydca.dca_utilities import dca_utilities


rna_msa_file = 'examples/MSA_RF00167.fa'
rna_refseq_file = 'examples/ref_RF00167.fa'

# create MSATrimmer instance 
trimmer = msa_trimmer.MSATrimmer(
    rna_msa_file, biomolecule='rna', 
    refseq_file=rna_refseq_file,
)

trimmed_data = trimmer.get_msa_trimmed_by_refseq(remove_all_gaps=True)

#write trimmed msa to file in FASTA format
trimmed_data_outfile = 'MSA_RF00167_Trimmed.fa'
with open(trimmed_data_outfile, 'w') as fh:
    for seqid, seq in trimmed_data:
        fh.write('>{}\n{}\n'.format(seqid, seq))
        


In the above, we have imported pydca or selected modules from pydca. Then we defined two variables `rna_msa_file` and `rna_refseq_file` that contain paths to an RNA MSA file and its reference sequence, respectively. Following that we created an MSATrimmer instance which enabled us to trim the MSA file. Finally, we write the trimmed MSA data to an output file `MSA_RF00167_Trimmed.fa` in FASTA format. To trim protein sequences we just need to change `biomolecule='rna'`  to `biomolecule='protein'` and provide the respective MSA and reference sequence files. Next we carry out DCA computations using the trimmed MSA saved in `MSA_RF00167_Trimmed.fa`

## DCA Computation Using Pseudolikelihood Maximization Algorithm (plmDCA)

In [8]:
# Compute DCA scores using Pseudolikelihood maximization algorithm

plmdca_inst = plmdca.PlmDCA(
    trimmed_data_outfile,
    'rna',
    seqid = 0.8,
    lambda_h = 1.0,
    lambda_J = 20.0,
    num_threads = 10,
    max_iterations = 500,
)

# compute DCA scores summarized by Frobenius norm and average product corrected
plmdca_FN_APC = plmdca_inst.compute_sorted_FN_APC()

In the above, we created a PlmDCA instance `plmdca_inst`  for RNA. Notice that we used the trimmed MSA data we obtained before.  We also set the values of optional parameters. The optional parameters `num_threads` is set to 10. If `pydca` is installed without OpenMP support, we cannot set the number of threads more than one. Finally we computed the DCA scores from the Frobenius norm of the couplings by calling the `compute_sorted_FN_APC()` method on `plmdca_inst`. This action returns the average product corrected (APC) DCA scores. Lets print the top five site pairs and their DCA scores.

In [5]:
for site_pair, score in plmdca_FN_APC[:5]:
    print(site_pair, score)

(45, 55) 2.5791187588802083
(44, 56) 2.5573842583919526
(16, 28) 2.5379174657520434
(17, 27) 2.5094812232835744
(43, 57) 2.4451864083984653


We have displayed the top five site pairs ranked by their DCA scores. The first column is a tuple of site-pairs and the second column contains the DCA scores obtained from the Frobenius norm of the couplings. Note that the site pairs are arranged in the form (i, j) such that j > i.

## DCA Computation Using Mean-Field Algorithm (mfDCA)

In [14]:
#create mean-field DCA instance 
mfdca_inst = meanfield_dca.MeanFieldDCA(
    trimmed_data_outfile,
    'rna',
    pseudocount = 0.5,
    seqid = 0.8,

)

# Compute average product corrected Frobenius norm of the couplings
mfdca_FN_APC = mfdca_inst.compute_sorted_FN_APC()

In the above, we created a mean-field DCA instance `mfdca_inst`. We set the optional parameters `pseudocount` and `seqid` to be `0.5` and `0.8`. Like in the plmDCA we computed the DCA scores from the Frobenius norm of the couplings and average product corrected. using `compute_sorted_FN_APC` metho of `mfdca_inst`. Lets print the top five ranked site pairs.

In [15]:
for site_pair, score in mfdca_FN_APC[:5]:
    print(site_pair, score)

(44, 56) 4.290005464965937
(16, 28) 4.210860173400807
(17, 27) 4.207758141824402
(45, 55) 4.190480172499375
(43, 57) 4.039065434604404
